# Experiment

## Graph Generation

This notebook is a way to reproduce the graphs used in our experiments in an interactive way.  First, we will get a gene-gene interaction graph.  Then, we will add the driver features from the `MERGE` algorithm as node features.

In [1]:
from data.gene_graphs import StringDBGraph
graph = StringDBGraph()

 loading from cache file/homes/gws/ewein/gene-expression/data/graphs/stringdb_graph_all_edges.adjlist


### MERGE Features

The following cell takes our gene interaction graph from above, and adds the MERGE features from each gene to the gene's respective node in the graph.

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

features_df = pd.read_csv("data/merge_features.csv")
features_df = features_df.drop('#SIGPVALS', axis=1)
features_df = features_df.drop('SCORE', axis=1)
features_df = features_df.set_index('GENE')
features_df = features_df.rename(
    columns={"meEthylation": "Methylation",
             "Genomic abnormalities (CNV)": "CNV",
             "Expression hub": "Hubness"})
features_df = features_df[~features_df.index.duplicated(keep='first')]

# Patient Gene + Response Data

We want to use the data from our gene interaction graph to 

In [3]:
import pickle

patient_data_dir = '/fdata/ohsu_data/'
patient_response_data = pickle.load(open(patient_data_dir+'final_frame.p','rb'))

patient_response_data = patient_response_data[['patient_id', 'new drug', "IC50"]]
drug = "Quizartinib"
drug_indices = patient_response_data["new drug"] == drug

# Some patients had multiple ex vivo measurements taken for a given drug.  We aggregate
# these measurements by taking the mean IC50 for each patient + drug combo
patient_response_data_for_drug = patient_response_data[patient_response_data["new drug"] == drug]
mean_drug_response = patient_response_data_for_drug.groupby("patient_id", as_index=False)['IC50'].mean()

## Gene Expression Data

Now we load the gene expression data for our patients from the cell above.  The original dataset repeats a given patient's measurements for each drug measurement taken.  We don't need the duplicates here, so we drop them.

In [4]:
rna_seq = pickle.load(open(patient_data_dir+'/X_rna_seq.p','rb'))
rna_seq = rna_seq.loc[:,~rna_seq.columns.duplicated()]

# Having the patient id for each entry (as opposed to just the numerical index) makes it easier
# to verify we're doing things correctly later on
rna_seq['patient_id'] = patient_response_data['patient_id']
rna_seq = rna_seq.drop_duplicates(subset='patient_id')
rna_seq = rna_seq.sort_values(by='patient_id')
rna_seq_for_drug = rna_seq[rna_seq.patient_id.isin(mean_drug_response.patient_id)]
rna_seq_for_drug = rna_seq_for_drug.drop('patient_id', axis=1)

### Subgraphing

Our sequencing data contains only a subset of the genes that are represented in our gene interaction graphs.  As such, we modify our interaction graph only to contain the nodes represented by the genes in our sequencing data.

In [5]:
genes = rna_seq_for_drug.columns
overlapping_genes = list(set(genes).intersection(features_df.index))
graph.nx_graph = graph.nx_graph.subgraph(overlapping_genes)

# Reorder our dataframes so that their genes have the same order as the nodes in our
# gene-gene interaction graph
rna_seq_for_drug = rna_seq_for_drug[list(graph.nx_graph.nodes)]
features_df = features_df.reindex(list(graph.nx_graph.nodes))

## Constructing the Dataset

### Input Data

Our input data $X \in \mathbb{R}^{N \times G \times D}$ is composed of $N$ patients, each with $G$ genes and $D$ features for each gene (one for expression level and one for each driver feature).  Here we merge the RNA expression data that we have with the driver feature data from above.

In [6]:
num_patients = rna_seq_for_drug.shape[0]
num_genes    = rna_seq_for_drug.shape[1]
num_features = features_df.shape[1] + 1 # Merge features plus one for gene expression level

X = np.ndarray(shape = (num_patients, num_genes, num_features), dtype = float)

for patient_index in range(num_patients):
    patient_rna_expression = rna_seq_for_drug.iloc[patient_index]
    new_patient = pd.DataFrame(index=patient_rna_expression.index, data={"Expression": patient_rna_expression})
    new_patient = pd.concat([new_patient, features_df], axis=1)
    X[patient_index] = new_patient


## Constructing the Normalized Edge Matrix

In [7]:
import scipy.sparse as sp
import torch

adj = sp.coo_matrix(nx.to_numpy_matrix(graph.nx_graph), dtype=np.float32)


adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

adj = normalize(adj + sp.eye(adj.shape[0]))
adj = sparse_mx_to_torch_sparse_tensor(adj).cuda()

### Output Data

Our output data $y \in \mathbb{R}^{N}$ is composed of drug response values for each of our $N$ patients.

In [8]:
Y = mean_drug_response["IC50"].to_numpy()

## Model Training

First we define the model.

In [9]:
from torch.nn import MSELoss
import torch.optim as optim
import torch
import datetime
import time
from IPython.core.debugger import set_trace
from models.models import GCN
from sklearn.model_selection import train_test_split

def train(model, num_epochs, X, Y, batch_size):
    # First split to 60% train, 40% test
    # Then further divide the original 40% test into 20% validation and 20% test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.8, random_state=42)
    #x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=0.5, random_state=42)
    
    
    #x_valid = torch.FloatTensor(x_valid).cuda()
    #y_valid = torch.FloatTensor(y_valid).cuda()
     
    x_test = torch.FloatTensor(x_test).cuda()
    y_test = torch.FloatTensor(y_test).cuda()
    
    print("Beginning model training at {}".format(datetime.datetime.now()))
    
    for epoch in range(num_epochs):
        start_time = time.time()
        for i in range(0, x_train.shape[0], batch_size):
            x_batch = torch.FloatTensor(x_train[i:i+batch_size]).cuda()
            y_batch = torch.FloatTensor(y_train[i:i+batch_size]).cuda()
            model.train()
            optimizer.zero_grad()
            
            # Convert batch_size x 1 tensor into 1d tensor of length batch_size
            output = model(x_batch, adj).squeeze(1)
            
            loss_train = criterion(output, y_batch)
            print("Epoch {} batch {}/{} train loss {}".format(epoch, i+batch_size, x_train.shape[0], loss_train))
            loss_train.backward()
            optimizer.step()
            
        model.eval()
        output = model(x_test, adj).squeeze(1)
        loss_test = criterion(output, y_test)
        end_time = time.time()
        epoch_time = end_time - start_time
        
        print("Epoch {} completed in {} secs with test loss {:.4f}".format(epoch, epoch_time, loss_test.item()))

In [10]:
gcn = GCN(
    nfeat = X.shape[2],
    nhid = 16,
    nnodes = X.shape[1],
    dropout = 0.5
).cuda()

criterion = MSELoss(reduction='mean')
optimizer = optim.Adam(gcn.parameters(), lr=1e-3, weight_decay=5e-4)

Then we train it!

In [ ]:
train(gcn, 100, X, Y, batch_size=32)

Beginning model training at 2019-10-10 12:54:31.473020
Epoch 0 batch 32/171 train loss 11.946531295776367
Epoch 0 batch 64/171 train loss 171288.71875
Epoch 0 batch 96/171 train loss 3455.5556640625
Epoch 0 batch 128/171 train loss 1220.82421875
Epoch 0 batch 160/171 train loss 1664.7615966796875
Epoch 0 batch 192/171 train loss 1439.8741455078125
Epoch 0 completed in 323.96815824508667 secs with test loss 1233.8590
Epoch 1 batch 32/171 train loss 1201.655029296875
Epoch 1 batch 64/171 train loss 886.487060546875
Epoch 1 batch 96/171 train loss 391.1154479980469
Epoch 1 batch 128/171 train loss 57.28619384765625
Epoch 1 batch 160/171 train loss 42.492523193359375
Epoch 1 batch 192/171 train loss 210.697265625
Epoch 1 completed in 322.37809228897095 secs with test loss 213.9976
Epoch 2 batch 32/171 train loss 245.49647521972656
Epoch 2 batch 64/171 train loss 253.244384765625
Epoch 2 batch 96/171 train loss 148.51190185546875
Epoch 2 batch 128/171 train loss 131.68734741210938
Epoch 2 b

Epoch 21 batch 192/171 train loss 3.9321792125701904
Epoch 21 completed in 324.6832346916199 secs with test loss 8.9152
Epoch 22 batch 32/171 train loss 9.954179763793945
Epoch 22 batch 64/171 train loss 15.610614776611328
Epoch 22 batch 96/171 train loss 17.423084259033203
Epoch 22 batch 128/171 train loss 8.092924118041992
Epoch 22 batch 160/171 train loss 9.562019348144531
Epoch 22 batch 192/171 train loss 7.328801155090332
Epoch 22 completed in 325.32474517822266 secs with test loss 8.8778
Epoch 23 batch 32/171 train loss 9.76272201538086
Epoch 23 batch 64/171 train loss 13.021763801574707
Epoch 23 batch 96/171 train loss 19.52459716796875
Epoch 23 batch 128/171 train loss 7.218841552734375
Epoch 23 batch 160/171 train loss 8.734879493713379
Epoch 23 batch 192/171 train loss 6.853228569030762
Epoch 23 completed in 325.46864461898804 secs with test loss 8.9816
Epoch 24 batch 32/171 train loss 9.775869369506836
Epoch 24 batch 64/171 train loss 9.763372421264648
Epoch 24 batch 96/171 

# Baselines

## MLP with Gene Expression

In [ ]:
from models.models import MLP
from torch.nn import MSELoss
from sklearn.model_selection import train_test_split
import datetime
import torch
import torch.optim as optim
import time

X = rna_seq_for_drug.values
Y = mean_drug_response["IC50"].to_numpy()


def train(model, num_epochs, X, Y, batch_size):
    # First split to 60% train, 40% test
    # Then further divide the original 40% test into 20% validation and 20% test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.8, random_state=42)
    #x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=0.5, random_state=42)
    
    
    #x_valid = torch.FloatTensor(x_valid).cuda()
    #y_valid = torch.FloatTensor(y_valid).cuda()
     
    x_test = torch.FloatTensor(x_test).cuda()
    y_test = torch.FloatTensor(y_test).cuda()
    
    print("Beginning model training at {}".format(datetime.datetime.now()))
    
    for epoch in range(num_epochs):
        start_time = time.time()
        for i in range(0, x_train.shape[0], batch_size):
            x_batch = torch.FloatTensor(x_train[i:i+batch_size]).cuda()
            y_batch = torch.FloatTensor(y_train[i:i+batch_size]).cuda()
            model.train()
            optimizer.zero_grad()
            
            # Convert batch_size x 1 tensor into 1d tensor of length batch_size
            output = model(x_batch).squeeze(1)
            
            loss_train = criterion(output, y_batch)
            print("Epoch {} batch {}/{} train loss {}".format(epoch, i+batch_size, x_train.shape[0], loss_train))
            loss_train.backward()
            optimizer.step()
            
        model.eval()
        output = model(x_test).squeeze(1)
        loss_test = criterion(output, y_test)
        end_time = time.time()
        epoch_time = end_time - start_time
        
        print("Epoch {} completed in {} secs with test loss {:.4f}".format(epoch, epoch_time, loss_test.item()))

In [ ]:
mlp = MLP(
    nnodes = X.shape[1],
    dropout = 0.5
).cuda()

criterion = MSELoss(reduction='mean')
optimizer = optim.Adam(mlp.parameters(),
                       lr=1e-5, weight_decay=5e-4)

train(mlp, 100, X, Y, 32)

## GCN only with Gene Expression Feature

In [ ]:
num_patients = rna_seq_for_drug.shape[0]
num_genes    = rna_seq_for_drug.shape[1]
num_features = 1 # Only care about gene expression level for the baseline

X = np.ndarray(shape = (num_patients, num_genes, num_features), dtype = float)

for patient_index in range(num_patients):
    patient_rna_expression = rna_seq_for_drug.iloc[patient_index]
    new_patient = pd.DataFrame(index=patient_rna_expression.index, data={"Expression": patient_rna_expression})
    X[patient_index] = new_patient

Train the baseline!

In [ ]:
from models.models import GCN

gcn_expression_only = GCN(
    nfeat = X.shape[2],
    nhid = 16,
    nnodes = X.shape[1],
    dropout = 0.5
).cuda()

criterion = MSELoss(reduction='mean')
optimizer = optim.Adam(gcn_expression_only.parameters(),
                       lr=1e-5, weight_decay=5e-4)

In [ ]:
Y = mean_drug_response["IC50"].to_numpy()

In [ ]:
train(gcn_expression_only, 100, X, Y, batch_size=32)

## ElasticNet Regression on Gene Expression Levels

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

num_patients = rna_seq_for_drug.shape[0]
num_genes    = rna_seq_for_drug.shape[1]
num_features = 1 # Only care about gene expression level for the baseline

X = np.ndarray(shape = (num_patients, num_genes, num_features), dtype = float)

for patient_index in range(num_patients):
    patient_rna_expression = rna_seq_for_drug.iloc[patient_index]
    new_patient = pd.DataFrame(index=patient_rna_expression.index, data={"Expression": patient_rna_expression})
    X[patient_index] = new_patient
    
X = X.squeeze(2)
y = patient_response_data_for_drug["IC50"].to_numpy()

train_valid_split=0.8
x_train, x_valid, y_train, y_valid = train_test_split(X, y, train_size=train_valid_split, test_size=1-train_valid_split)

In [ ]:
def train_elasticnet_with_params(alpha, l1_ratio):
    regr = ElasticNet(random_state=0, alpha = alpha, l1_ratio = l1_ratio)
    regr.fit(x_train, y_train)
    
    y_pred = regr.predict(x_valid)
    mse = np.mean(np.square(y_pred - y_valid))
    print("Test error {:.4f} for alpha={:.4f} and l1_ratio={:.4f}".format(mse, alpha, l1_ratio))
    return mse

In [ ]:
import itertools

alpha_vals = np.linspace(start = 0, stop = 5, num = 20)
l1_ratio = np.linspace(start = 0, stop = 1.0, num = 20)

best_mse = float("inf")
for alpha, l1_ratio in itertools.product(alpha_vals, l1_ratio):
    mse_new = train_elasticnet_with_params(alpha=alpha, l1_ratio=l1_ratio)
    if mse_new < best_mse:
        best_mse = mse_new

print("Best elastic net MSE: {:.4f}".format(best_mse))